In [ ]:
%matplotlib notebook
#%matplotlib inline

import matplotlib.pyplot as plt
import rasterio
import rasterio.warp
import numpy as np
import math
from scipy import ndimage as ndi

import pyproj
import xarray as xr
import fiona
import shapely
import pydash
from osgeo import osr
from numpy import pi, r_, c_, vstack, hstack
import sys
import cv2

import kk.dggs
from kk.dggs import DGGS
from kk.tools import polygon_path

In [ ]:
def get_affine(fname):
    with rasterio.open(fname) as f:
        return f.affine

def get_poly_lines():
    with fiona.open('/Users/kirill/wk/dggs/qgis/data/cstauscd_l.shp') as f:
        return [np.r_[ pydash.get(p, 'geometry.coordinates') ].T 
                for p in f if pydash.get(p, 'properties.FEAT_CODE') == 'tile_edge']

In [ ]:
filename='/Users/kirill/wk/dggs/earth.tif'

In [ ]:
A = get_affine(filename)
ds = xr.open_rasterio(filename)
ds.attrs['affine'] = A
src = np.dstack(ds.values)
src_ = src[::8,::8,:]

In [ ]:
A, ds.crs, ds.attrs

In [ ]:
pp_tile_edge = get_poly_lines()
len(pp_tile_edge)    

In [ ]:
plt.figure(figsize=(9,5))
plt.imshow(src_, extent=[-180,180,-90,90])
for p in pp_tile_edge:
    x,y = p
    plt.plot(x,y,'m-')
#plt.axis('equal')
pass

In [ ]:
dg = DGGS()

In [ ]:
def warp_all(ds, dg, scale_level):
    def impl(code):
        addr = code + '0'*scale_level
        wrp = dg.mk_warper(addr)
        bands = [wrp(ds.isel(band=i).values, ds.affine) for i in range(ds.band.shape[0])]
        
        return addr, bands
    
    return dict(map(impl, 'SNOPQR'))
        
bb = warp_all(ds, dg, scale_level=7)      

In [ ]:
def merge_extents(e1, e2):
    if e1 is None:
        return e2
    
    return [min(e1[0], e2[0]), max(e1[1], e2[1]),
            min(e1[2], e2[2]), max(e1[3], e2[3])]

def plot_all(south_square=0, north_square=0):
    dh = dg.mk_display_helper(south_square=south_square, north_square=north_square)
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    extents = None

    for a, b in bb.items():
        im, ee = dh(a, np.dstack(b))
        #print(ee)
        extents = merge_extents(extents, ee)
        ax.imshow(im, extent=ee)

    ax.set_xlim(*extents[:2])
    ax.set_ylim(*extents[2:])
    return fig

fig = plot_all(south_square=3, north_square=2)

In [ ]:
dh = dg.mk_display_helper(south_square=3, north_square=2)

[dh(code+'340', (123,37)) for code in 'SNOPQR']

In [ ]:
def cell_extent(cell_addr):
    tr, (maxW, maxH) = dg.pixel_coord_transform(cell_addr)

    b = 0.5 - 1e-6
    x,y = polygon_path([-b,+b])

    return tr(x,y)

def plot_cell(cell_addr, style='-', ax = None):
    if ax is None:
        ax = plt.gca()
        
    u,v = cell_extent(cell_addr)
    return ax.plot(u,v, style)
    
fig = plt.figure(figsize=(9,5))
ax = fig.add_subplot(111)
ax.imshow(src_, extent=(-180,180,-90,90))

plot_cell('R', 'r-')
plot_cell('R65', 'c-')
plot_cell('R68', 'c-')
plot_cell('R7', 'm-')
plot_cell('R7444444', 'm-')



#plot_cell('S6', 'y-')
for i in range(9):
    if i not in [0,4,8]:
        plot_cell('S0'+str(i), 'm-')
    
    plot_cell('S6'+str(i), 'y-')
    plot_cell('S2'+str(i), 'c-')
    plot_cell('S8'+str(i), 'm-')

    plot_cell('S3'+str(i), 'g-')
    plot_cell('S1'+str(i), 'r-')
    plot_cell('S7'+str(i), 'b-')
    plot_cell('S5'+str(i), 'w-')

ax.axis('equal')
pass

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.Axes(fig, [0,0, 1,1])
ax.set_axis_off()
ax.set_xlim(0,4)
ax.set_ylim(0,3)
fig.add_axes(ax)

ex = lambda x,y: (x, x + 1, y, y + 1)

ax.imshow(np.dstack(bb['N0000000']), extent=ex(0,2))
ax.imshow(np.dstack(bb['O0000000']), extent=ex(0,1))
ax.imshow(np.dstack(bb['S0000000']), extent=ex(0,0))
ax.imshow(np.dstack(bb['P0000000']), extent=ex(1,1))
ax.imshow(np.dstack(bb['Q0000000']), extent=ex(2,1))
ax.imshow(np.dstack(bb['R0000000']), extent=ex(3,1))